# BasicTick: Create Cluster: HDB for basictickdb
This notebook will create (start) an HDB cluster on a named basictick database.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *

# Managed KX Database and Cluster names to create
DB_NAME="basictickdb"

SEC_THREADS='8'

BASE_NAME="HDB"
CLUSTER_NAME=f"{BASE_NAME}_{DB_NAME}_20230601"

# Cluster Settings
CODEBASE="basictick"
S3_CODE_PATH="code"

DB_PATHS = [ '/' ]

CAPACITY_CONFIG={ 'nodeCount': 3, 'nodeType': 'kx.s.xlarge'}
DATABASE_CONFIG=[{ 'databaseName': DB_NAME,'cacheConfigurations': [{'dbPaths': DB_PATHS, 'cacheType': 'CACHE_1000' }] }]
CACHE_CONFIG=[{'type': 'CACHE_1000', 'size':1200}]

CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/hdbmkdb.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE} 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Check Database

In [4]:
note_str = ""

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: basictickdb, Changesets: 3 
Changeset (COMPLETED): ZsQ3eXNny4UaexiXtmEkZg: Created: 2023-05-30 23:22:00.273000+00:00


changeType,s3Path,dbPath
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


Changeset (COMPLETED): QsQ3lbQfnmumBgZOeiPIOw: Created: 2023-05-31 00:18:15.744000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/W9tLqnobb65iKp72sWNF1/0/,/2023.05.31/
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/W9tLqnobb65iKp72sWNF1/1/,/


Changeset (COMPLETED): uMQ5qhWLaIRqY97nyAU9cg: Created: 2023-05-31 19:41:15.927000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/57-boC3sloCP4ZeZkw2ql/0/,/2023.05.31/
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/57-boC3sloCP4ZeZkw2ql/1/,/


# Deploy code

In [5]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE} -x '*.ipynb_checkpoints*'       
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE} -x '*.ipynb_checkpoints*'   
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/query.q (deflated 19%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/connectmkdb.q (deflated 63%)
upload: ./basictick.zip to s3://kdb-demo-612841383594/code/basictick.zip
2023-06-01 11:25:49   68132277 ManagedKdbInsights-main.zip
2023-06-01 14:17:10      28372 basictick.zip
2023-05-31 15:23:45        757 code.zip
2023-05-31 10:19:36        652 taqcode.zip
2023-05-31 10:24:55        787 welcomedb.zip


0

## Create Cluster

In [6]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='HDB',
    releaseLabel = '1.0',
    capacityConfiguration=CAPACITY_CONFIG,
    databases=DATABASE_CONFIG,
    cacheStorageConfigurations=CACHE_CONFIG,
    clusterDescription="Created with create_HDB for basic_tick notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [7]:
resp

{'ResponseMetadata': {'RequestId': '169f7a40-eef5-4a53-8557-8e76f297f719',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1141',
   'connection': 'keep-alive',
   'date': 'Thu, 01 Jun 2023 14:17:17 GMT',
   'x-amzn-requestid': '169f7a40-eef5-4a53-8557-8e76f297f719',
   'x-amz-apigw-id': 'F18_-FeIoAMF64A=',
   'x-amzn-trace-id': 'Root=1-6478a865-63dedc0746f546b412fc6c69',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 0003b3450f3f9fac44312c4622a410c2.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'e2keNma6S0_Jcaeu8BiF9_EMcPEOwCo7cufwCmwKpXpepLZOOjLGDQ=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'HDB_basictickdb_20230601',
 'clusterType': 'HDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': 'uMQ5qhWLaIRqY97nyAU9cg'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size': 120

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: HDB_basictickdb_20230601 status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: HDB_basictickdb_20230601 status is CREATING, total wait 0:05:0

In [9]:
# Give permissions time to propogate after cluster creation....
time.sleep(30)

In [10]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-088aaeb956ebd6d92-l09z9oen.vpce-svc-038b591b42a835849.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-088aaeb956ebd6d92-l09z9oen.vpce-svc-038b591b42a835849.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEIf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIEYkBQsm%2Fo8qACPLwVlX7hTX9PtSjkwQJjDmrrppme7wAiA43o4ZRr%2BUL1bSqyVWc5RT3n6Ig4TkYXHdB53JHWlrSir%2FAgjA%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAAaDDYxMjg0MTM4MzU5NCIMTjMK2M2d9Uk8wlZWKtMCVYvMpFaJ6s5bDwhd%2BGIpoaVtQsdketMl7b5Wzuj6s7o7USnYNy9zElomZYXdmzh18eGSOb5vLOr6j9MICM1vhM4mVynCC11BAFCQFzl%2B5Fi%2BkzVBv%2FT1InWMYhTHbjE6jrj3lD%2BGT9M5wB%2Fmc8cZR4kHnT0PmwUs14tuU6%2Bee9kkuFnY4Tp1zLEZpZQYQ%2BW9SSvl6GSJdF6AAA25E%2F5j9jbCNqTGovF1THZ3Yi55CDtC%2F79S1lnwsBMBgVxEalB53Ogs8w%2F%2BLdhslpcL7sTrfr9NH2o2fGQr95eOVViYi6BXE%2BTPoY5M2BhIzV7JSpfhshoAHJ6XVKgVJAsYkA94vEm0bWncV9k4bT0d8bR4xiXoR7MA0mN4B%2FKUO8sa8HKnrAbaBvYjDyAkFtNcdpszGSBB99G1uMwRVr5uYoQHsPnF6OAssQbjE9Eu%2BbQEuOAP

In [11]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
HDB_TAQ_2021H1,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '32'}, {'key': 'dbname', 'value': 'TAQ_2021H1'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_H1 notebook,2023-05-31 10:40:45.063000+00:00,2023-05-31 10:19:39.315000+00:00,TAQ_2021H1,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/', '/2021.01.06/', '/2021.01.07/', '/2021.01.08/', '/2021.01.11/', '/2021.01.12/', '/2021.01.13/', '/2021.01.14/', '/2021.01.15/', '/2021.01.19/', '/2021.01.20/', '/2021.01.21/', '/2021.01.22/', '/2021.01.25/', '/2021.01.26/', '/2021.01.27/', '/2021.01.28/', '/2021.01.29/', '/2021.02.01/', '/2021.02.02/', '/2021.02.03/', '/2021.02.04/', '/2021.02.05/']}]"
HDB_TAQ_2021_2D,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'TAQ_2021_2D'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_2D notebook,2023-05-31 10:40:17.015000+00:00,2023-05-31 10:19:34.957000+00:00,TAQ_2021_2D,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/']}]"
HDB_basictickdb_20230601,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}]",Created with create_HDB for basic_tick notebook,2023-06-01 14:36:42.745000+00:00,2023-06-01 14:17:16.753000+00:00,basictickdb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
HDB_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_HDB notebook,2023-05-30 23:13:20.948000+00:00,2023-05-30 22:56:11.773000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
RDB_basictickdb_20230601,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant2'}]",Created with create_RDB notebook,2023-06-01 14:15:36.253000+00:00,2023-06-01 14:01:48.077000+00:00,basictickdb,[]
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-05-30 23:10:28.477000+00:00,2023-05-30 22:56:52.812000+00:00,welcomedb,[]
cluster_create_delete_db,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'create_delete_db'}, {'key': 'codebase', 'value': 'code'}]",Demo Cluster for database create_delete_db,2023-05-31 15:42:40.298000+00:00,2023-05-31 15:23:52.245000+00:00,create_delete_db,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
cluster_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'welcomedb'}]",Demo Cluster for database welcomedb,2023-05-30 23:10:58.551000+00:00,2023-05-30 22:53:31.356000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


In [12]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-01 14:37:36.771394
